In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.metrics import roc_curve,roc_auc_score,auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from catboost import Pool, CatBoostClassifier, cv

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-mar-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-mar-2021/test.csv')
sample = pd.read_csv('../input/tabular-playground-series-mar-2021/sample_submission.csv')

In [ ]:
train.drop('id', axis = 1, inplace=True)
test.drop('id', axis = 1, inplace=True)

In [ ]:
cat_features = [
    "cat0", "cat1", "cat2", "cat3", "cat4", "cat5", "cat6", "cat7",
    "cat8", "cat9", "cat10", "cat11", "cat12", "cat13", "cat14", "cat15",
    "cat16", "cat17", "cat18"
]

In [ ]:
for col in cat_features:
    
    le = LabelEncoder()

    le.fit(train[col].unique().tolist() + test[col].unique().tolist())

    train[col] = le.transform(train[col])
    test[col] = le.transform(test[col])

In [ ]:
y = train['target']

X = train.drop('target', axis = 1)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15, random_state=1, stratify=y)

In [ ]:
cb = CatBoostClassifier(
        verbose=200,
        eval_metric="AUC",
        loss_function="Logloss",
        random_state=1,
        num_boost_round=15000,
        task_type="GPU",
        thread_count=-1
    )

In [ ]:
cb.fit(X_train, y_train, plot=True, use_best_model=True,eval_set=(X_val, y_val))

In [ ]:
train_pred = cb.predict_proba(X_train,thread_count=-1)
y_pred = cb.predict_proba(test,thread_count=-1)
val_pred = cb.predict_proba(X_val,thread_count=-1)

In [ ]:
train_pred = train_pred[:,1]
y_pred = y_pred[:,1]
val_pred = val_pred[:,1]

In [ ]:
print("AUC ROC train   ", roc_auc_score(y_train, train_pred))
print("AUC ROC val     ", roc_auc_score(y_val, val_pred))

In [ ]:
sample['target'] = y_pred
sample

In [ ]:
sample.to_csv("cb_baseline.csv",index=False)